In [9]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)

file = "newANDfail.csv"
df = pd.read_csv(file)
# df.head()

for index, row in df.iterrows():

    real_output = eval(str(row['real_output']))
    df.at[index, 'Real Output'] = real_output['n200']

    gold_output = eval(str(row['gold_output']))
    df.at[index, 'Gold Output'] = gold_output['n200']
    
    pv = str(row['MC']).split(',')
    df.at[index, 'Process Variation'] = pv[1]

    df.at[index, 'Equal?'] = (gold_output['n200'] == real_output['n200'])

drop_col = ['MC', 'real_output', 'gold_output']
df = df.drop(drop_col, axis=1)

df = df[df['Equal?'] == False]
# df.head(10)

variations = df['Process Variation'].unique()


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\583293288.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00000010000000000010000000000010000000000010100100100110' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Real Output'] = real_output['n200']
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\583293288.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00000001000000000001000000000001000000000001010010010011' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Gold Output'] = gold_output['n200']
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\583293288.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.0' has dtype incompatible wit

In [10]:
from functions_file import *

pd.set_option('display.max_columns', None)
fault_distribution = pd.DataFrame()

for pv in variations:

    sample = df[df['Process Variation'] == pv]


    for index, row in sample.iterrows():
        realOutput = str(row['Real Output'])
        goldOutput = str(row['Gold Output'])
        inputPattern = str(row['pattern']).replace(",", "")

        sample.loc[index, 'SA0'] = 1 if O_SA0(realOutput) else 0
        sample.at[index, 'SA1'] = 1 if O_SA1(realOutput) else 0
        sample.at[index, 'EscapeRight'] = 1 if outputEscapeRight(goldOutput, realOutput) else 0
        sample.at[index, 'EscapeLeft'] = 1 if outputEscapeLeft(goldOutput, realOutput) else 0
        sample.at[index, 'Overflow'] = 1 if overflow(realOutput) else 0
        sample.at[index, 'AND->OR'] = 1 if AND_OR(inputPattern, realOutput) else 0
        sample.at[index, 'OR->AND'] = 1 if OR_AND(inputPattern, realOutput) else 0
        sample.at[index, 'alwaysFail_00'] = 1 if alwaysFail_00(inputPattern, realOutput, goldOutput) else 0
        sample.at[index, 'alwaysFail_01'] = 1 if alwaysFail_01(inputPattern, realOutput, goldOutput) else 0
        sample.at[index, 'alwaysFail_10'] = 1 if alwaysFail_10(inputPattern, realOutput, goldOutput) else 0
        sample.at[index, 'alwaysFail_11'] = 1 if alwaysFail_11(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_00'] = 1 if oftenFail_00(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_01'] = 1 if oftenFail_01(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_10'] = 1 if oftenFail_10(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_11'] = 1 if oftenFail_11(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_00'] = 1 if neverFail_00(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_01'] = 1 if neverFail_01(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_10'] = 1 if neverFail_10(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_11'] = 1 if neverFail_11(inputPattern, realOutput, goldOutput) else 0

        for sequence in sequences:
            if alwaysFail_sequence(inputPattern, realOutput, goldOutput, sequence) is True:
                sample.at[index, "alwaysFail_" + str(sequence)] = 1
            else:
                sample.at[index, "alwaysFail_" + str(sequence)] = 0
        

    sample['totalTemplates'] = sample.iloc[:, 6:sample.shape[1]-1].sum(axis=1)

    new_row = {}
    for col in sample.columns:
        if pd.api.types.is_numeric_dtype(sample[col]):
            new_row[col] = sample[col].sum()
        else:
            new_row[col] = sample[col].iloc[-1]

    
    sample = pd.concat([sample, pd.DataFrame([new_row], columns=sample.columns)], ignore_index=True)

    print("Process Variation: ", pv)
    display(sample.head())
    display(sample.tail())

    fault_distribution = pd.concat([fault_distribution, pd.DataFrame([sample.iloc[-1]], columns=sample.columns)], ignore_index=True)

C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.loc[index, 'SA0'] = 1 if O_SA0(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.at[index, 'SA1'] = 1 if O_SA1(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Process Variation:  -0.0


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['totalTemplates'] = sample.iloc[:, 6:sample.shape[1]-1].sum(axis=1)


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
0,and_spb_0_and_spb_0_and_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,and_spb_0_and_spb_0_and_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,and_spb_0_and_spb_0_and_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,and_spb_0_and_spb_0_and_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,and_spb_0_and_spb_0_and_spa_0_dff,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
731,xor_spb_3_xor_spb_3_and_spa_3_dff,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
732,xor_spb_3_xor_spb_3_and_spa_3_inv,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
733,xor_spb_3_xor_spb_3_and_spa_3_or,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
734,xor_spb_3_xor_spb_3_and_spa_3_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
735,xor_spb_3_xor_spb_3_and_spa_3_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,-0.0,False,0.0,0.0,20.0,550.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,580.0


...

C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.loc[index, 'SA0'] = 1 if O_SA0(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.at[index, 'SA1'] = 1 if O_SA1(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Process Variation:  0.02


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['totalTemplates'] = sample.iloc[:, 6:sample.shape[1]-1].sum(axis=1)


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
0,and_spb_0_and_spb_0_and_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,0.02,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,and_spb_0_and_spb_0_and_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,0.02,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,and_spb_0_and_spb_0_and_spa_0_dff,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,0.02,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,and_spb_0_and_spb_0_and_spa_0_dff,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,0.02,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,and_spb_0_and_spb_0_and_spa_0_inv,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000010000000000010000000000010000000000010100100100110,00000001000000000001000000000001000000000001010010010011,0.02,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
1996,xor_spb_3_xor_spb_3_and_spa_3_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,0.02,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,xor_spb_3_xor_spb_3_and_spa_3_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,0.02,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,xor_spb_3_xor_spb_3_and_spa_3_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,0.02,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999,xor_spb_3_xor_spb_3_and_spa_3_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,0.02,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000,xor_spb_3_xor_spb_3_and_spa_3_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000000100000000000100000000000000000000000001001001001,00000001000000000001000000000001000000000001010010010011,0.02,False,45.0,0.0,240.0,875.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1240.0


In [11]:
drop_cols = ['config', 'pattern', 'Real Output', 'Gold Output', 'Equal?']
fault_distribution = fault_distribution.drop(drop_cols, axis=1).sort_values(by='Process Variation')
fault_distribution

,Process Variation,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
0,-0.0,0.0,0.0,20.0,550.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,580.0
1,-0.01,35.0,0.0,840.0,1455.0,0.0,0.0,0.0,0.0,0.0,0.0,197.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2527.0
7,-0.02,45.0,0.0,470.0,1750.0,0.0,0.0,0.0,0.0,0.0,0.0,185.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,2455.0
4,-0.03,125.0,0.0,40.0,990.0,0.0,0.0,0.0,0.0,0.0,0.0,215.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1370.0
5,-0.04,165.0,0.0,180.0,1170.0,0.0,0.0,0.0,0.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1760.0
11,-0.05,80.0,0.0,625.0,1610.0,0.0,0.0,0.0,0.0,0.0,0.0,294.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,2665.0
2,-0.06,170.0,0.0,445.0,1855.0,0.0,0.0,0.0,0.0,0.0,0.0,405.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2876.0
17,-0.07,195.0,0.0,320.0,830.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1630.0
3,-0.08,0.0,0.0,190.0,905.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1180.0
18,-0.09,125.0,0.0,60.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,135.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,830.0


In [12]:
fault_distribution_ratio = fault_distribution.iloc[:, 1:-1].div(fault_distribution['totalTemplates'], axis=0)
fault_distribution_ratio.insert(0, 'Process Variation', fault_distribution['Process Variation'])
fault_distribution_ratio


,Process Variation,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111
0,-0.0,0.000000,0.0,0.034483,0.948276,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.008621
1,-0.01,0.013850,0.0,0.332410,0.575782,0.0,0.0,0.0,0.0,0.0,0.0,0.077958,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7,-0.02,0.018330,0.0,0.191446,0.712831,0.0,0.0,0.0,0.0,0.0,0.0,0.075356,0.0,0.0,0.0,0.002037,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.038697
4,-0.03,0.091241,0.0,0.029197,0.722628,0.0,0.0,0.0,0.0,0.0,0.0,0.156934,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
5,-0.04,0.093750,0.0,0.102273,0.664773,0.0,0.0,0.0,0.0,0.0,0.0,0.136364,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.002841,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
11,-0.05,0.030019,0.0,0.234522,0.604128,0.0,0.0,0.0,0.0,0.0,0.0,0.110319,0.0,0.0,0.0,0.021013,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.031895
2,-0.06,0.059110,0.0,0.154729,0.644993,0.0,0.0,0.0,0.0,0.0,0.0,0.140821,0.0,0.0,0.0,0.000348,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
17,-0.07,0.119632,0.0,0.196319,0.509202,0.0,0.0,0.0,0.0,0.0,0.0,0.171779,0.0,0.0,0.0,0.003067,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.003067
3,-0.08,0.000000,0.0,0.161017,0.766949,0.0,0.0,0.0,0.0,0.0,0.0,0.072034,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
18,-0.09,0.150602,0.0,0.072289,0.602410,0.0,0.0,0.0,0.0,0.0,0.0,0.162651,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.012048,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
